<a href="https://colab.research.google.com/github/rajaranjith/HCL-GenAI-Training/blob/main/Assignment_1_Gold_Badge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install dependencies
# Install the official Mistral Python SDK
!pip install -q mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.


In [3]:
#Set up API key and client
import os
import getpass
from mistralai import Mistral

# Ask for your API key once per Colab session
if "MISTRAL_API_KEY" not in os.environ or not os.environ["MISTRAL_API_KEY"]:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

print("✅ Mistral client initialized.")

Enter your Mistral API key: ··········
✅ Mistral client initialized.


In [13]:
#Create a tiny sentiment dataset and save as JSONL
import json

# Tiny toy dataset: (text, label)
train_examples = [
    ("I love this phone, the battery lasts all day.", "positive"),
    ("Terrible experience. The screen broke in two days.", "negative"),
    ("Amazing camera and very fast performance!", "positive"),
    ("The app keeps crashing, I'm very disappointed.", "negative"),
    ("Works fine but the sound quality could be better.", "negative"),
    ("Best purchase I've made this year.", "positive"),
    ("Really happy with this laptop overall.", "positive"),
    ("Customer support was rude and unhelpful.", "negative"),
    ("This headset is comfortable and sounds great.", "positive"),
    ("The product stopped working after a week.", "negative"),
]

eval_examples = [
    ("Pretty good overall, just a bit heavy.", "positive"),
    ("Worst customer service I've ever seen.", "negative"),
    ("Battery life is awful and it overheats.", "negative"),
    ("I’m impressed, it works better than expected.", "positive"),
]

INSTRUCTION = (
    "You are a sentiment classifier.\n"
    "Reply with EXACTLY one word: 'positive' or 'negative'.\n"
    "No explanations, no punctuation, just the label.\n\n"
    "Text: "
)

def write_jsonl(filename, pairs):
    with open(filename, "w", encoding="utf-8") as f:
        for text, label in pairs:
            record = {
                "messages": [
                    {
                        "role": "user",
                        "content": INSTRUCTION + text,
                    },
                    {
                        "role": "assistant",
                        "content": label,
                    },
                ]
            }
            # Each JSON object must be on a single line
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

write_jsonl("sentiment_train.jsonl", train_examples)
write_jsonl("sentiment_eval.jsonl", eval_examples)

print("✅ Wrote sentiment_train.jsonl and sentiment_eval.jsonl")

# Quick sanity check – show first 2 lines
!head -n 2 sentiment_train.jsonl

✅ Wrote sentiment_train.jsonl and sentiment_eval.jsonl
{"messages": [{"role": "user", "content": "You are a sentiment classifier.\nReply with EXACTLY one word: 'positive' or 'negative'.\nNo explanations, no punctuation, just the label.\n\nText: I love this phone, the battery lasts all day."}, {"role": "assistant", "content": "positive"}]}
{"messages": [{"role": "user", "content": "You are a sentiment classifier.\nReply with EXACTLY one word: 'positive' or 'negative'.\nNo explanations, no punctuation, just the label.\n\nText: Terrible experience. The screen broke in two days."}, {"role": "assistant", "content": "negative"}]}


In [14]:
#Upload dataset files to Mistral
# Upload the training and validation files to Mistral Cloud
train_file = client.files.upload(
    file={
        "file_name": "sentiment_train.jsonl",
        "content": open("sentiment_train.jsonl", "rb"),
    }
)

eval_file = client.files.upload(
    file={
        "file_name": "sentiment_eval.jsonl",
        "content": open("sentiment_eval.jsonl", "rb"),
    }
)

print("✅ Uploaded files.")
print("Training file id:", train_file.id)
print("Eval file id    :", eval_file.id)

✅ Uploaded files.
Training file id: 662aa371-3e8d-45dd-9362-466d63444fab
Eval file id    : 7da21861-80ee-4f7e-98f0-2f2e650a9417


In [21]:
#(Optional) Pretty-print file metadata
import json

def pprint_obj(o):
    print(json.dumps(o.model_dump(), indent=2))

print("Training file:")
pprint_obj(train_file)

print("\nValidation file:")
pprint_obj(eval_file)

Training file:
{
  "id": "662aa371-3e8d-45dd-9362-466d63444fab",
  "object": "file",
  "bytes": null,
  "created_at": 1766567464,
  "filename": "sentiment_train.jsonl",
  "purpose": "fine-tune",
  "sample_type": "instruct",
  "source": "upload",
  "num_lines": 10,
  "mimetype": "application/jsonl",
  "signature": "6f8cfef570603e54"
}

Validation file:
{
  "id": "7da21861-80ee-4f7e-98f0-2f2e650a9417",
  "object": "file",
  "bytes": null,
  "created_at": 1766567464,
  "filename": "sentiment_eval.jsonl",
  "purpose": "fine-tune",
  "sample_type": "instruct",
  "source": "upload",
  "num_lines": 4,
  "mimetype": "application/jsonl",
  "signature": "f5f94968f7dd93d0"
}


In [64]:
#Create a fine-tuning job
hyperparams = {
    "training_steps": 50,   # small for demo
    "learning_rate": 1e-4,
}

job = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": train_file.id, "weight": 1}],
    validation_files=[eval_file.id],
    hyperparameters=hyperparams,
    auto_start=True,  # start as soon as it validates
)

print("✅ Created fine-tuning job.")
print("Job id:", job.id)

# Optional: inspect full job object
import json
print(json.dumps(job.model_dump(), indent=2))

✅ Created fine-tuning job.
Job id: 8cbadd93-1d10-4c16-a1b0-ccc8361878dd
{
  "id": "8cbadd93-1d10-4c16-a1b0-ccc8361878dd",
  "auto_start": true,
  "model": "open-mistral-7b",
  "status": "QUEUED",
  "created_at": 1766574153,
  "modified_at": 1766574153,
  "training_files": [
    "662aa371-3e8d-45dd-9362-466d63444fab"
  ],
  "hyperparameters": {
    "training_steps": 50,
    "learning_rate": 0.0001,
    "weight_decay": 0.1,
    "warmup_fraction": 0.05,
    "epochs": null,
    "seq_len": 32768,
    "fim_ratio": null
  },
  "validation_files": [
    "7da21861-80ee-4f7e-98f0-2f2e650a9417"
  ],
  "object": "job",
  "fine_tuned_model": null,
  "suffix": null,
  "integrations": [],
  "trained_tokens": null,
  "metadata": {
    "expected_duration_seconds": null,
    "cost": 0.0,
    "cost_currency": null,
    "train_tokens_per_step": null,
    "train_tokens": null,
    "data_tokens": null,
    "estimated_start_time": null
  },
  "job_type": "completion",
  "repositories": []
}


In [66]:
#Poll job status until it finishes
import time
import json

def pprint_obj(o):
    print(json.dumps(o.model_dump(), indent=2))

job_id = job.id

retrieved_job = client.fine_tuning.jobs.get(job_id=job_id)
print("Initial status:", retrieved_job.status)

while retrieved_job.status in ("QUEUED", "VALIDATED", "RUNNING"):
    pprint_obj(retrieved_job)
    print(f"Status is {retrieved_job.status} – checking again in 10 seconds...\n")
    time.sleep(10)
    retrieved_job = client.fine_tuning.jobs.get(job_id=job_id)

print("🚩 Final status:", retrieved_job.status)
if retrieved_job.status == "SUCCEEDED":
    print("🎉 Fine-tuned model:", retrieved_job.fine_tuned_model)
else:
    print("Job did not succeed – check job details above.")

Initial status: FAILED_VALIDATION
🚩 Final status: FAILED_VALIDATION
Job did not succeed – check job details above.


In [24]:
#Use the fine-tuned model for inference
# Make sure the job actually succeeded
if retrieved_job.status != "SUCCEEDED":
    raise RuntimeError(f"Job status is {retrieved_job.status}, cannot run inference.")

fine_tuned_model = retrieved_job.fine_tuned_model
print("Using model:", fine_tuned_model)

test_texts = [
    "This is the best thing I've ever bought.",
    "Horrible quality, I'm never buying from them again.",
    "Pretty decent for the price.",
    "I hate how slow and buggy this is.",
]

for text in test_texts:
    resp = client.chat.complete(
        model=fine_tuned_model,
        messages=[
            {
                "role": "user",
                "content": INSTRUCTION + text,
            }
        ],
        # optional: make responses deterministic
        temperature=0.0,
    )

    # Safely extract text from the response via .dict()
    resp_data = resp.dict()
    content_blocks = resp_data["choices"][0]["message"]["content"]
    # Concatenate all text blocks (there's usually just one)
    prediction = "".join(block.get("text", "") for block in content_blocks).strip()

    print(f"\nText: {text}")
    print("Predicted label:", prediction)

RuntimeError: Job status is FAILED_VALIDATION, cannot run inference.